## User-defined functions
`funcdef` :== [`<decorators>`] `def` `<funcname>` [`<type_params>`] `(` [`<parameter_list>`] `)` [`->` `<expression>`]:\
    `<suite>` 

`parameter_list` :== [`<position_only_parameters>`, `\`] `,` `<position_or_keyword_params>` `,` [`<keyword_only_params>`]

`parameter` =   `<identifier>` [`=` `<expression>`]

## Parameter Resolution
1. A parameter may be resolved by its position or by keyword.
2. By default if both `\` and `*` are not present then both position and keyword paramters can be provided. 
3. Position parameters are formal parameters who whose arguments are decided by their position from left to right
4. Keyword parameters are formal params whose arguments are decided by keyword.
5. Default parameters can be either positional or keyword and carry a default value which will be used if no argument is supplied at runtime.
6. The default value is an object, as everything else in python, and is set at compile time. Its not evaluated again at runtime. This should be understood as it may have effects if an mutable object is used as default value.
7. A special formal parameter of the kinf pointer to an object can be specified to hold any extra positional arguments supplied
8. A special formal paramter of the kind pointer to an array of pointers to object can be specified to hold all extra keyword parameters supplied. 
9. A function can return a value. 

#### There are many rules that govern the usage of paramters. We will progressively build and examine these rules.

#### Passing arguments to positional parameters
1. This is the most basic usage, something that we all have used many times in all programming languages.
2. These are USED AS positional parameters and evaluated from left to right. So formal param `p1` gets `10` and `p2` gets `str`
3. Note the 'USED AS' above. We can use them as keyword parameters also.

In [5]:
def func(p1,p2): 
    print(f'{p1=}, {p2=}')

func(10,'str') # treated as positional params


p1=10, p2='str'


### Passing arguments to keyword paramters
1. The same function can be passed arguments considering them as keywords

In [8]:
def func(p1, p2): 
    print(f"{p1=}, {p2=}")

func(p1=10, p2="str")  # treated as keyword params

p1=10, p2='str'


##### But what if we want them passed as positional only??
### Postional only params
1. There is a special parameter `/` that expects no value.
2. Its purpose is to say - 'All formal params before me are strictly positional'

##### Why do we need them??
1. Many built-in functions are implemented in C and they do not accept keyword arguments. This allows us to be consistent with C code.
2. Some python methods (of a class) get the first argument as 'self'. This needs to be alwasys positional and is passed by Python internally. You will encounter this pattern very often with the constructor of a class.
3. At times there is no inherent meaning to the names of a parameter. Consider a function `add(num1, num2)`. There is no need to have the developer pass keyowrd params. In fact, to a reader its a lot more intuitive to think of the function as adding any 2 arbitary numbers. 

In [10]:
def func(p1, p2, /):
    print(f"{p1=}, {p2=}")

# func(p1=10, p2="str")  # Gives TypeError = we cannot now pass keyword params.

func(10,'str') # this works. 

p1=10, p2='str'


##### What if we want keyword only params.
### Keyword only parameters
1. There is a special parameter `*` that expects no value
2. Presence of this parameter says 'From here on all parameters are keyword only'

##### But why do you we need them?
1. Allows us to reorder the paramters.
2. We can alter the signature and add some default params without breaking existing code.
3. Works great for functions that supports options.
4. There are cases where we use a '*args' pattern to cat all left over positional arguments. This needs to be specified after all positional arguments are filled. There is another patters `**kwargs` to catch all left over keyword args. And it to needs to be specified last. If keyword only params construct was not there we will have to specify both `*args` and `**kwargs` and then extract.  

In [12]:
def func(*, p1, p2):
    print(f"{p1=}, {p2=}")

# func(10, 'str') # throws TypeError 

func(p1 = 10, p2 = 'str') # this works.




p1=10, p2='str'


### Having positional only, keyword only and some params that can be both positional and keyword
1. Use the `/` to first specify positional only params.
2. Then specify params that can be both positional and keyword
3. Use the '*' param to specify start of keyword only params.

In [15]:
def func(p1,/, pk1, *, k1):
    print(f"{p1=}, {pk1=}, {k1=}")

func('positional', 'both', k1 = 'kw') # works
func("positional", pk1 = "both", k1="kw")  # works

p1='positional', pk1='both', k1='kw'
p1='positional', pk1='both', k1='kw'


### Positional arguments cannot appear after keyword arguments.
1. When calling a function that has multiple postional and keyword parameters then positional argument cannot be specified after a keyword one

In [20]:
def func(p1, /, pk1, pk2, *, k1):
    print(f"{p1=}, {pk1=}, {pk2=}, {k1=}")


# func("positional", pk1= "both",10,  k1="kw")  # does not work
func("positional",  "both",pk2=10,  k1="kw")  # works

p1='positional', pk1='both', pk2=10, k1='kw'


### Catch all 'left over' positional params
1. At times the number of arguments that a function accepts is variable. 
2. The the catch all param for left over postional params is `*paramname`
3. All the left overs are packed together in a tuple.
4. The catch all should be the last param.
5. A positional only seperator `/` is not allowed after the catch all. Its really not needed and no positional param can be provided after a positional catch all. All params post a positional only catch all are considered keyword only.
6. In fact the carch all is nothing but the `*` param without a name. So when you want to ensure that random extra positional arguments should not be supplied we use `*` without a name. Else we give it a name and use that to reference all left over positional arguments.  

In [27]:
def func1(p1, *allPositionals):
    print(f'{p1=}, {allPositionals=}')

def func2(*allPositionals, p1 ): # Here p1 is positional on.ly
    print(f"{p1=}, {allPositionals=}")

func1(10,23,'str') 

p1=10, allPositionals=(23, 'str')


##### what if we have positional only and then a variable number of both positional and keyword followed by a few keyword options.

In [28]:
def func(p1,/,pk1, *args, k1):
    print(f'{p1=}, {pk1=}, {args=}, {k1=}')

# func(10, pk1=23, 34, "str", k1="key")  # wont work as positional cannot come after keyword.
func(10, 23, 34, "str", k1="key")  # works.

p1=10, pk1=23, args=(34, 'str'), k1='key'


### Catch all left over keyword arguments.
1. The parameter has the pattern `**paramname`
2. It obviously comes after all positional params and is the last keyword param
3. No parameter can follow it
4. They are packed into a dict.

In [35]:
def func(*args, **kwargs): # fully left over
    print(f'{args=}, {kwargs=}')

func(10,23,34,k1=10,k2='str')


def func(pk1, *args, **kwargs): # one positional/keyword followed by left overs
    print(f"{pk1=}, {args=}, {kwargs=}")

func(pk1=10,  k1=10, k2="str") # since positional cannot come after keyword, if we call like this args will also be empty

func(10, 23, 34, k2="str") # works


def func(pk1, *args,k1, **kwargs):  # one positional/keyword followed by left overs
    print(f"{pk1=}, {args=}, {k1=} {kwargs=}")

func(10, 23, 34,k1='k1', k2="str")  # works

def func(**kwargs):
    print(f'{kwargs}')

#func(10,k2=10) # Gives error as the func signature does not allow any positional arguments. 

func(k1=10, k2='str') # works.

args=(10, 23, 34), kwargs={'k1': 10, 'k2': 'str'}
pk1=10, args=(), kwargs={'k1': 10, 'k2': 'str'}
pk1=10, args=(23, 34), kwargs={'k2': 'str'}
pk1=10, args=(23, 34), k1='k1' kwargs={'k2': 'str'}
{'k1': 10, 'k2': 'str'}


### Default Value
1. Any param can be provided a defualt value.
2. If a positional parameter is provided a default value then post that all positional parameters mmust have a default value.
3. In the case of keyword only params this rule does not apply as the order is irrelevant with them

In [41]:
def func(*, p1=10, p2, **kwargs):
    print(f'{p1=}, {p2=}, {kwargs=}')

func(p1=12, p2='str', p3='extra')

def func(p1, p2=10):
    print(f"{p1=}, {p2=}")

# func(p1=12, p2="str", p3="extra") # extra params not allowed as catch all not present.

func(p1=12, p2="str")

# def func(p1, p2=10, p3): # Wont work as non-default cannot follow default positional arguement.


def func(p1, p2=10, *, p3): # this works as the special param for keyword only is added before p3.
    pass

p1=12, p2='str', kwargs={'p3': 'extra'}
p1=12, p2='str'


### Function with mutable default value
1. The default value of the function is set up in memory when the function is defined. Before its called.
2. Subsequent calls do not alter the default value and the same object is used in subsequent calls that rely on default value
3. This can have certain effects that we need to be aware of in case the default value is mutable and we mutate it in the function

In [48]:
def func(p1=[1,2,3]):
    p1.append(10)
    print(id(p1), p1) # the id of the object remains same across calls.

func()
func()

4437508800 [1, 2, 3, 10]
4437508800 [1, 2, 3, 10, 10]


### Returning values from a function

In [49]:
# Returning Multiple Values
def get_dimensions():
    return 10, 20  # Returning a tuple

print(get_dimensions())  # returns a tuple.


(10, 20)


### Annotations
1. Python annotations are used to indicate the data type
2. There are 2 main types of annotations - function annotations and variable annotation.
3. They are not binding on Python and are just for the aid of the developer.
4. There are however libraries that can be used in projects that enforce the types specified in annotations.

In [55]:
# Annotations
def add(a: int, b: int) -> int:
    return a + b

# add() is expected to take integers and return an integer

# access annotation
print(add.__annotations__)  # {'a': <class 'int'>, 'b': <class 'int'>, 'return': <class 'int'>}
print(f"{add(1,4)=}") # takes in and returns an int.
print(f"{add('str1','str2')=}") # takes strings and returns a string.



{'a': <class 'int'>, 'b': <class 'int'>, 'return': <class 'int'>}
()
add(1,4)=5
add('str1','str2')='str1str2'


### Generic Functions
1. Lets say you want to define a function with annotations for controlling the type of params and return value
2. But your function supports multiple types of parameters - what will you do? Surely you dont want to write a separate function for each type.
3. This is where generics come in.

In [62]:
def func[T](p1: T, p2: T) -> T: # A generic function
    retVal = None
    if isinstance(p1, int): # check if both are integers
        retVal = p1 + p2
    if isinstance(p1, list): # check for list
        p1.extend(p2)
        retVal = p1
    return retVal

print(func.__type_params__) # gives a list of all types defined by the function
print(f'{func(1,2)=}')
print(f'{func([1,2], [3,4])=}')

(T,)
func(1,2)=3
func([1,2], [3,4])=[1, 2, 3, 4]


1. What the above code does during compilation is create an annotation scope which is not bound to any name!
2. Within this scope a new function is defined say TYPE_PARAMS_OF_func(). Where the trailing 'func' is the name of out function
2. In this function the type `T` is created: `T = typing.TypeVar("T")`
3. Next a nested function thats exactly the same as our function is created
4. The nested the value of `__type_params__` is also set in 
5. Finally the outer function returns the nested function (our original function)

`annotation-def TYPE_PARAMS_OF_func():`\
    `T = typing.TypeVar("T")`\
    `def func(arg: T): ...`\
    `func.__type_params__ = (T,)`\
    `return func`\
`func = TYPE_PARAMS_OF_func()`

### Functions as first class citizens
1. Functions are objects and therefore can be assigned to a variable.
2. They can be assigned to a variable
3. Passes as params to a function
4. Returned from a function
5. Can be packed in a composite data type like a list.

In [10]:
# Assigning Functions to Variables

def greet():
    print("Hello!")

# Assign the greet function to a new variable
greeting = greet

# Call the function using the new variable
greeting()  # Output: Hello!


Hello!


In [11]:
# Passing a Function as an Argument

def call_function(fn):
    # Call the function passed as an argument
    fn()

def say_hello():
    print("Hello from say_hello!")

# Pass say_hello as an argument
call_function(say_hello)  # Output: Hello from say_hello!


Hello from say_hello!


In [12]:
# Returning a Function from Another Function

def outer_function():
    def inner_function():
        print("Hello from the inner function!")
    
    return inner_function  # Return the inner function

# Get the inner function
my_function = outer_function()

# Call the returned function
my_function()  # Output: Hello from the inner function!


Hello from the inner function!


In [13]:
# Storing Functions in Data Structures

def add(x, y):
    return x + y

def subtract(x, y):
    return x - y

# Store functions in a list
operations = [add, subtract]

# Call functions from the list
print(operations[0](10, 5))  # Output: 15 (calls add(10, 5))
print(operations[1](10, 5))  # Output: 5  (calls subtract(10, 5))


15
5


In [14]:
# Function Attributes

def my_function():
    print("Hello!")

# Function's name
print(my_function.__name__)  # Output: my_function

# Global variables accessible in the function's scope
print(my_function.__globals__)  # Prints global variables in the module


my_function
{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', '# Defining a Function\ndef greet(name):\n    return f"Hello, {name}!"\n\n# Calling the Function\nmessage = greet("Jayant")\nprint(message)  # Output: Hello, Jayant!', '# Returning Multiple Values\ndef get_dimensions():\n    return 10, 20  # Returning a tuple\n\nlength, width = get_dimensions()\nprint(length, width)  # 10 20', '# Formal vs. Actual Parameters\n\n# Formal parameters: a, b\ndef add(a, b):\n    return a + b\n\n# Actual parameters: 3, 4\nresult = add(3, 4)', '# Formal vs. Actual Parameters\n\n# Formal parameters: a, b\ndef add(a, b):\n    return a + b\n\n# Actual parameters: 3, 4\nresult = add(3, 4)', "# Annotations\ndef add(a: int, b: int) -> int:\n    return a + b\n\n# add() is expected to take 

### Anonymous Functions with lambda

In [15]:
add = lambda x, y: x + y
print(add(3, 4))  # Output: 7


7


In [16]:
# Example with map
numbers = [1, 2, 3, 4]
squares = list(map(lambda x: x**2, numbers))
print(squares)  # Output: [1, 4, 9, 16]


[1, 4, 9, 16]


### Built-In functions
1. These are functions provided by Python interpreter
2. They are always available in any scope
3. We look at a few useful ones here.
4. A more comprehensive list is here: https://docs.python.org/3.13/library/functions.html

#### callable()
1. Returns boolean - True if the object _appears_ callable.  
2. Any object that implements the `__call__()` method is callable.
3. The presence of this method does not guarantee that the call will be successful.

In [67]:
# callable()

def func():
    pass

print(f'{callable(func)=}') # A function is callable.

class A:
    def __call__():
        print('Call method of the class')

print(f'{callable(A)=}')

a = A()

print(f"{callable(a)=}")


class B:
    pass # the  __call__ is not implemented

print(f"{callable(B)=}") # will be true as classes are always callable and return a new instance.

b = B()

print(f"{callable(b)=}") # this is false now.

val: int = 10
print(f"{callable(val)=}") # this is obviously false.

callable(func)=True
callable(A)=True
callable(a)=True
callable(B)=True
callable(b)=False
callable(val)=False


#### dir() & dir(x)
1. When used without an argument it rerutns the list of all names in the local scope.
2. With argument, returns the a list of valid attributes of the object. 

In [71]:
val = 1
def func():
    val2 = 2
    print(dir()) # print all the attributes local to function

func()
dir(func) # print all the attributes of the function object.


['val2']


['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__type_params__']

#### eval()
